In [71]:
import torch
import torch.nn as nn
import math
import re
from torch.utils.data import Dataset, DataLoader
import collections

class CosineSimilarity(nn.Module):
    def forward(self, tensor_1,tensor_2):
        normalized_tensor_1=tensor_1/tensor_1.norm(dim=-1,keepdim=True)
        normalized_tensor_2=tensor_2/tensor_2.norm(dim=-1,keepdim=True)
        return (normalized_tensor_1*normalized_tensor_2).sum(dim=-1)

In [73]:
#for test
input1 =  torch.randn(1,200,requires_grad=True)
input2 =  torch.randn(1,200,requires_grad=True)
con    =  CosineSimilarity()
s1     =  con(input1,input2)
s2     =  torch.cosine_similarity(input1,input1)
print(s1)
print(s2)

tensor([-0.0021], grad_fn=<SumBackward1>)
tensor([1.0000], grad_fn=<SumBackward1>)


In [74]:
# dealwith text
def tokenize(text):
    fileters = ['!', '"', '#', '$', '%', '&', '\(', '\)', '\*', '\+', ',', '-', '\.', '/', ':', ';', '<', '=', '>', '\?', '@'
        , '\[', '\\', '\]', '^', '_', '`', '\{', '\|', '\}', '~', '\t', '\n', '\x97', '\x96', '”', '“', '0', '1', '2',
               '3', '4', '5', '6', '7', '8', '9', '10']
    text = re.sub("<.*?>", " ", text, flags=re.S)
    text = re.sub("|".join(fileters), " ", text, flags=re.S)
    return [i.strip() for i in text.split()]

In [75]:
#test 
res = tokenize("a3dnjsdf99099_")
print(res)

['a', 'dnjsdf']


In [124]:
#gen dict
def gendict(x,y):
    z  = x + '!' + y
    t  = tokenize(z)
    li = list(set(t))
    print('diction:')
    print(li)
    tinydict = {}
    it = 1;
    for i in li:
        tinydict[i] = it
        it = it + 1
    for i in li:
        tinydict[i] = tinydict[i]/(len(li)+1)
    print('kv:')
    print(tinydict)
    xx = []
    yy = []
    for ix in tokenize(x):
        xx.append(tinydict[ix])
    for iy in tokenize(y):
        yy.append(tinydict[iy])
    #print(xx)
    #print(yy)
    zz = (xx,yy)
    #print('text to number:')
    #print(zz)
    return (xx,yy)


In [125]:
#test gendict
z = gendict("a3dnjsdf99099_","a3dnjsdf99099aa_")
print(z)

diction:
['a', 'aa', 'dnjsdf']
kv:
{'a': 0.25, 'aa': 0.5, 'dnjsdf': 0.75}
([0.25, 0.75], [0.25, 0.75, 0.5])


In [126]:
#f(x,y)
def f(x,y):
    dic_x = tokenize(x)
    dic_x_count = len(dic_x)
    print(dic_x)
    dic_y = tokenize(y)
    dic_y_count = len(dic_y)
    print(dic_y)
    sx = 0.0
    sy = 0.0
    for xx in dic_x:
        if xx in dic_y:
            sx = sx + 1.0/dic_x_count
    for yy in dic_y:
        if yy in dic_x:
            sy = sy + 1.0/dic_y_count
    print(sx)
    print(sy)
    return max(sx,sy)

In [127]:
#test f(x,y)
x = "3dqn1n1j1s99099_"
y = "3dqn1js99099_"
test_f = f(x,y)
print(test_f)

['dqn', 'n', 'j', 's']
['dqn', 'js']
0.25
0.5
0.5


In [128]:
def embed(x,y):
    dic_x = tokenize(x)
    dic_x_count = len(dic_x)
    dic_y = tokenize(y)
    dic_y_count = len(dic_y)
    emb = torch.nn.Embedding(dic_x_count + dic_y_count,5)
    

In [132]:
#g(x,y)
def g(x,y):
    (xx,yy) = gendict(x,y)
    count = min(len(xx),len(yy))
    print(count)
    tx = []
    ty = []
    for i in range(0,count):
        tx.append(xx[i])
        ty.append(yy[i])
    print(tx)
    print(ty)
    return con(torch.tensor(tx),torch.tensor(ty))

In [134]:
#test g(x,y)
x = "3dqn1n1j1s99099_"
y = "3dqn1js99099_"
test_g = g(x,y)
print(test_g)

diction:
['j', 'n', 's', 'js', 'dqn']
kv:
{'j': 0.16666666666666666, 'n': 0.3333333333333333, 's': 0.5, 'js': 0.6666666666666666, 'dqn': 0.8333333333333334}
2
[0.8333333333333334, 0.3333333333333333]
[0.8333333333333334, 0.6666666666666666]
tensor(0.9570)


In [144]:
#result
x = "int sum(int x,int y){return x - y;}"
y = "int summary(float z){return z;}"
s = max(f(x,y), g(x,y))
print(s)

['int', 'sum', 'int', 'x', 'int', 'y', 'return', 'x', 'y']
['int', 'summary', 'float', 'z', 'return', 'z']
0.4444444444444444
0.3333333333333333
diction:
['y', 'return', 'x', 'summary', 'sum', 'int', 'z', 'float']
kv:
{'y': 0.1111111111111111, 'return': 0.2222222222222222, 'x': 0.3333333333333333, 'summary': 0.4444444444444444, 'sum': 0.5555555555555556, 'int': 0.6666666666666666, 'z': 0.7777777777777778, 'float': 0.8888888888888888}
6
[0.6666666666666666, 0.5555555555555556, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.1111111111111111]
[0.6666666666666666, 0.4444444444444444, 0.8888888888888888, 0.7777777777777778, 0.2222222222222222, 0.7777777777777778]
tensor(0.8156)
